# 0. Prep the Environment

In [ ]:
import os, gzip, glob, shutil, pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, f_oneway
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import plotly.express as px
import seaborn as sns
from sklearn import decomposition

In [ ]:
for gz_fn in glob.glob('../data/*.gz'):
  if not os.path.exists(gz_fn[:-3]):
    with gzip.open(gz_fn, 'rb') as f_in, open(gz_fn[:-3], 'wb') as f_out:
      shutil.copyfileobj(f_in, f_out)

# 1. Parse File (Remove in Solution Notebook)

In [ ]:
with open('../data/GSE8685_family.soft') as f:
    lines=f.readlines()
    
    on_expression_table=False
    on_platform_table=False
    on_sample=False
    
    sample=''
    sample_key={}
    
    probe_key={}    
    expression_table={}
    p_val_table={}
    transcript_table={}
    abs_call_table={}
    
    for line in lines:
        
        line=line.strip()
        
        if line=='!platform_table_begin':
            on_platform_table=True
            continue
            
        elif line=='!platform_table_end':
            on_platform_table=False
            continue
            
        elif line=='!sample_table_begin':
            on_expression_table=True
            continue
            
        elif line=='!sample_table_end':
            on_expression_table=False
            continue
            
        elif line.split(' = ')[0]=='^SAMPLE':
            sample=line.split(' = ')[1]
            expression_table[sample]={}
            transcript_table[sample]={}
            p_val_table[sample]={}
            abs_call_table[sample]={}
            continue
            
        elif line.split(' = ')[0]=='!Sample_title':
            sample_key[sample]=line.split(' = ')[1]
            continue
        
        line=line.strip().split('\t')
        
        if on_platform_table:
            if len(line)<10:
                if line[5]=='':
                    print(line)
                probe_key[line[0]]=line[5]
                continue
                
            else:
                gene_ids=line[10].split(' /// ')

                probe_key[line[0]]=gene_ids
                continue
            
        elif on_expression_table:
            if sample=='':
                print(line)
            if line[0]=='ID_REF':
                continue
            else:
                value=line[1]
                p_val=line[3]
                abs_call=line[2]
                probe_id=line[0]
                gene_names=probe_key[probe_id]
                if type(gene_names)==list:
                    for gene_name in gene_names:
                        if not gene_name in expression_table[sample]:
                            expression_table[sample][gene_name]=[float(value)]
                        else:
                            expression_table[sample][gene_name].append(float(value))
                        
                else:
                    if not gene_names in expression_table[sample]:
                        expression_table[sample][gene_names]=[float(value)]
                    else:
                        expression_table[sample][gene_names].append(float(value))
                p_val_table[sample][probe_id]=float(p_val) 
                abs_call_table[sample][probe_id]= 1 if abs_call=='P' else 0
                transcript_table[sample][probe_id]=float(value)

In [ ]:
sample_key

In [ ]:
sample_key_abb={
    'GSM215347':'Control 1',
    'GSM215348':'Control 2',
    'GSM215349':'Control 3',
    'GSM215350':'Activated with IL2 rep 1',
    'GSM215351':'Activated with IL2 rep 2',
    'GSM215352':'Activated with IL2 rep 3',
    'GSM215353':'Activated with IL15 rep 1',
    'GSM215354':'Activated with IL15 rep 2',
    'GSM215355':'Activated with IL15 rep 3',
    'GSM215356':'Activated with IL21 rep 1',
    'GSM215357':'Activated with IL21 rep 2',
    'GSM215358':'Activated with IL21 rep 3'
}

experimental_groups={
    'Control': ['GSM215347', 'GSM215348', 'GSM215349'],
    'IL2': ['GSM215350', 'GSM215351', 'GSM215352'],
    'IL15': ['GSM215353', 'GSM215354', 'GSM215355'],
    'IL21': ['GSM215356', 'GSM215357', 'GSM215358'],
}



In [ ]:
transcript_df=pd.DataFrame(transcript_table).dropna()

In [ ]:
transcript_df

# 2. Retain only transcripts with present calls in all members of at least one condition

In [ ]:
abs_call_df=pd.DataFrame(abs_call_table)
abs_call_df

In [ ]:
present_transcripts=set()

for transcript in abs_call_df.index:
    for condition in experimental_groups:
        
        if abs_call_df.loc[transcript, experimental_groups[condition]].sum()==3:
            present_transcripts.add(transcript)
            break

In [ ]:
len(present_transcripts)

In [ ]:
present_transcripts_df=transcript_df.loc[present_transcripts]
present_transcripts_df.shape

# 3. PCA

In [ ]:
pca=decomposition.PCA(n_components=3)
components=pca.fit_transform(present_transcripts_df.T.values)
components_df=pd.DataFrame(components, index=present_transcripts_df.T.index)

#### We want a high combined explained variance ratio. If there we can't get a high explained variance ratio within the first 2-3 components, we should choose a different method

In [ ]:
pca.explained_variance_ratio_

In [ ]:
pca.explained_variance_ratio_.sum()

In [ ]:
components_df

### 3D Scatterplot - visualize PCA

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
experimental_groups

In [ ]:
fig=plt.figure(figsize=(10,10), dpi=100)
ax=fig.gca(projection='3d')
for i, j in zip(experimental_groups, 'rygb'):
    xs=components_df.loc[experimental_groups[i], 0].values
    ys=components_df.loc[experimental_groups[i], 1].values
    zs=components_df.loc[experimental_groups[i], 2]
    ax.scatter(xs, ys, zs, c=j, s=400)
    
ax.legend(labels=['Control', 'IL2', 'IL15', 'IL21'])
plt.show()
fig.savefig('../results/3d_pca.png')

In [ ]:
# Make a bunch of 2D scatterplots instead of one 3D scatterplot

for combo in [(0,1), (0,2), (1,2)]:
    fig=plt.figure(figsize=(5,5), dpi=100)
    ax=fig.gca()
    for i, j in zip(experimental_groups, 'rygb'):
        xs=components_df.loc[experimental_groups[i], combo[0]].values
        ys=components_df.loc[experimental_groups[i], combo[1]].values
    
        ax.scatter(xs, ys, c=j, s=400)
        fig.savefig('../results/2d_pca_{}.png'.format(combo))

# 4. One-way ANOVA/F-test on 4 conditions

In [ ]:
experimental_groups_expression={}
for condition in experimental_groups:
    experimental_groups_expression[condition]=present_transcripts_df[experimental_groups[condition]].T

In [ ]:
F, p=f_oneway(experimental_groups_expression['Control'], 
               experimental_groups_expression['IL2'],
               experimental_groups_expression['IL15'],
               experimental_groups_expression['IL21'], axis=0)

In [ ]:
len(F)

In [ ]:
transcripts=list(present_transcripts_df.index)

In [ ]:
F_p_df=pd.DataFrame(index=transcripts, data=zip(F, p), columns=['F', 'p'])

In [ ]:
F_p_df

# 5. Benjamini-Hochberg step up correction

In [ ]:
bh=multipletests(p, alpha=0.01, method='fdr_bh')
F_p_df['reject null']=bh[0]
F_p_df['corrected p']=bh[1]

In [ ]:
diff_exp_genes_df=F_p_df.loc[F_p_df['reject null']]

In [ ]:
diff_exp_genes_list=list(diff_exp_genes_df.index)

In [ ]:
len(diff_exp_genes_list)

# 5. Diff-exp A: Filter by fold change (No post-hoc)

## Calculating fold change vs control

In [ ]:
#Calculate mean by experimental group
condition_means={}
condition_means_diffexp={}

for i in experimental_groups:
    condition_means_diffexp[i]=present_transcripts_df.loc[diff_exp_genes_list,experimental_groups[i]].T.mean()
    condition_means[i]=present_transcripts_df.loc[:, experimental_groups[i]].T.mean()    
    
condition_means_df=pd.DataFrame(condition_means)
condition_means_diffexp_df=pd.DataFrame(condition_means_diffexp)

In [ ]:
present_transcripts_df.loc[:, experimental_groups['IL2']].T.mean()/present_transcripts_df.loc[:, experimental_groups['Control']].T.mean() 

In [ ]:
#Calculate fold change
fold_change_dict={}
for i in condition_means_df:
    fold_change_dict[i]=condition_means_df[i]/condition_means_df['Control']
fold_change_df=pd.DataFrame(fold_change_dict)

In [ ]:
fold_change_df

## Filter for fold change magnitude

In [ ]:
diffexp_fold_change={}

for i in fold_change_df:
    
    if not i=='Control':
        diffexp_fold_change[i]={'up':[],
                      'dn':[]}
        
        #Filter for significance
        for j in diff_exp_genes_list:
            
            fold_change=fold_change_df.loc[j, i]
            
            if fold_change > 2:
                diffexp_fold_change[i]['up'].append(j)
                
            elif fold_change < 0.5:
                diffexp_fold_change[i]['dn'].append(j)

In [ ]:
print("Differentially expressed probes in each group:")
for i in diffexp_fold_change:
    for j in diffexp_fold_change[i]:
        print ('{}_{}: {}'.format(i, j, len(diffexp_fold_change[i][j])))

In [ ]:
#Change probe names to gene names

diffexp_genes_fold_change={}
for i in diffexp_fold_change:
    diffexp_genes_fold_change[i]={}
    for j in diffexp_fold_change[i]:
        diffexp_genes_fold_change[i][j]=set()
        for probe in diffexp_fold_change[i][j]:
            if type(probe_key[probe])==list:
                for gene in probe_key[probe]:
                    diffexp_genes_fold_change[i][j].add(gene)

In [ ]:
print("Differentially expressed genes in each group")

for i in diffexp_genes_fold_change:
    for j in diffexp_genes_fold_change[i]:
        print('{}_{}: {}'.format(i, j, len(diffexp_genes_fold_change[i][j])))

In [ ]:
from matplotlib_venn import venn3

In [ ]:
venn_dict=dict(zip(diffexp_fold_change.keys(), [diffexp_fold_change[i]['up']+diffexp_fold_change[i]['dn'] for i in diffexp_fold_change]))

up_venn_dict=dict(zip(diffexp_fold_change.keys(), [diffexp_fold_change[i]['up'] for i in diffexp_fold_change]))
dn_venn_dict=dict(zip(diffexp_fold_change.keys(), [diffexp_fold_change[i]['dn'] for i in diffexp_fold_change]))

In [ ]:
venn3(subsets=[set(venn_dict[condition]) for condition in venn_dict], set_labels=venn_dict.keys())

In [ ]:
venn3(subsets=[set(up_venn_dict[condition]) for condition in venn_dict], set_labels=venn_dict.keys())

In [ ]:
venn3(subsets=[set(dn_venn_dict[condition]) for condition in venn_dict], set_labels=venn_dict.keys())

# Calculate Pearson Correlation

In [ ]:
#Create pearson correlation coefficient matrix between columns of a matrix
def matrix_pearson(df):
    pearsons={}
    pvals={}
    for column0 in df:
        pearsons[column0]=[]
        pvals[column0]=[]
        for column1 in df:
                r, p=pearsonr(df[column0], df[column1])
                pearsons[column0].append(r)
                pvals[column0].append(p)
    return(pd.DataFrame(pearsons), pd.DataFrame(pvals, dtype=float))

In [ ]:
pearson, p=matrix_pearson(present_transcripts_df)

In [ ]:
df=pd.DataFrame(np.random.randint(0,100, size=(100,6)), columns=list('abcdef'))

In [ ]:
matrix_pearson(df)

In [ ]:
pearson.columns=[sample_key_abb[i] for i in pearson.columns]

In [ ]:
pearson.index=pearson.columns

In [ ]:
pearson

In [ ]:
sns.heatmap(pearson)
plt.savefig('../results/Pearson_clustermap.png')

# Compare with Gene Sets

In [ ]:
gene_sets={
    'GSE8685_IL15_ACT_IL2_STARVED_VS_IL21_ACT_IL2_STARVED_CD4_TCELL_UP': 'IL15_IL21_up',
    'GSE8685_IL15_ACT_IL2_STARVED_VS_IL21_ACT_IL2_STARVED_CD4_TCELL_DN': 'IL15_IL21_dn',
    'GSE8685_IL2_ACT_IL2_STARVED_VS_IL15_ACT_IL2_STARVED_CD4_TCELL_DN': 'IL15_Control_dn',
    'GSE8685_IL2_ACT_IL2_STARVED_VS_IL15_ACT_IL2_STARVED_CD4_TCELL_UP': 'IL15_Control_up',
    'GSE8685_IL2_ACT_IL2_STARVED_VS_IL21_ACT_IL2_STARVED_CD4_TCELL_DN': 'IL2_IL21_dn',
    'GSE8685_IL2_ACT_IL2_STARVED_VS_IL21_ACT_IL2_STARVED_CD4_TCELL_UP': 'IL2_IL21_up',
    'GSE8685_IL2_STARVED_VS_IL15_ACT_IL2_STARVED_CD4_TCELL_DN': 'IL15_Control_dn',
    'GSE8685_IL2_STARVED_VS_IL15_ACT_IL2_STARVED_CD4_TCELL_UP': 'IL15_Control_up',
    'GSE8685_IL2_STARVED_VS_IL21_ACT_IL2_STARVED_CD4_TCELL_DN': 'IL21_Control_dn',
    'GSE8685_IL2_STARVED_VS_IL21_ACT_IL2_STARVED_CD4_TCELL_UP': 'IL21_Control_up',
    'GSE8685_IL2_STARVED_VS_IL2_ACT_IL2_STARVED_CD4_TCELL_DN': 'IL2_Control_dn',
    'GSE8685_IL2_STARVED_VS_IL2_ACT_IL2_STARVED_CD4_TCELL_UP': 'IL2_Control_up',
    'IL15_UP.V1_DN':'IL15_Control_dn',
    'IL15_UP.V1_UP':'IL15_Control_up',
    'IL21_UP.V1_DN':'IL21_Control_dn',
    'IL21_UP.V1_UP':'IL21_Control_up',
    'IL2_UP.V1_DN':'IL2_Control_dn',
    'IL2_UP.V1_UP':'IL2_Control_up',
    'MARZEC_IL2_SIGNALING_DN': 'IL2_Control_dn',
    'MARZEC_IL2_SIGNALING_UP':'IL2_Control_up',
}

In [ ]:
gene_sets_dict=pickle.load(open('../data/msigdb_marzec_gene_sets.p', 'rb'))

In [ ]:
from matplotlib_venn import venn2

In [ ]:
for gene_set in gene_sets:
    condition=gene_sets[gene_set].split('_')
    up_or_dn=condition[2]
    if condition[1] == 'Control':
        gene_set_genes=set(gene_sets_dict[gene_set])
        diff_exp_genes=set(diffexp_genes_fold_change[condition[0]][up_or_dn])
        print(gene_sets[gene_set])
        fig=plt.figure()
        ax=plt.gca()
        venn2([gene_set_genes, diff_exp_genes], set_labels=[gene_set, 'our genes'], ax=ax)

# Diff-exp B: Post-Hoc Analysis by using Pairwise Tukey

In [ ]:
def tukeyhsd_get_significant_comparisons(tukeyhsd_results, include_CI=True, only_sig_comparisons=True):
    
    summary_df=pd.DataFrame(tukeyhsd_results._results_table.data[1:], columns=tukeyhsd_results._results_table.data[0])
    
    desired_comparisons = [True] if only_sig_comparisons else [True, False]    
    
    a=summary_df.loc[:,['p-adj', 'reject']]
    
    if include_CI:
        
        a['CI'] = [[i[0] for i in zip(summary_df.loc[j, ['lower', 'upper']])] for j in summary_df.index]
        
    a.index=[i+'_'+j for i,j in summary_df.loc[:,['group1', 'group2']].values]
        
    return a.loc[[True if i in desired_comparisons else False for i in summary_df['reject']]]

In [ ]:
post_hoc_diff_dict={}
for transcript in diff_exp_genes_list:
    
    tukey_summary=pairwise_tukeyhsd(present_transcripts_df.loc[transcript].to_numpy(), ['Control']*3+['IL2']*3+['IL15']*3+['IL21']*3, alpha=0.01)
    tukey_results=tukeyhsd_get_significant_comparisons(tukey_summary, include_CI=True)
    
    for comparison in tukey_results.index:
        if not comparison in post_hoc_diff_dict:
            post_hoc_diff_dict[comparison]={}
        post_hoc_diff_dict[comparison][transcript]=tukey_results.loc[comparison, 'p-adj']

In [ ]:
post_hoc_diff_df=pd.DataFrame(post_hoc_diff_dict)

In [ ]:
print("Significant differentially expressed genes after post hoc analysis:")

for condition in post_hoc_diff_dict:
    print('{}: {} genes'.format(condition, len(post_hoc_diff_dict[condition])))

### Filter for Fold Change 2 or higher

In [ ]:
post_hoc_fold_change_dict={}
for comparison in post_hoc_diff_dict:
    condA=comparison.split('_')[0]
    condB=comparison.split('_')[1]
    post_hoc_fold_change_dict[comparison]=present_transcripts_df.loc[:,experimental_groups[condB]].T.mean()/present_transcripts_df.loc[:,experimental_groups[condA]].T.mean()
post_hoc_fold_change_df=pd.DataFrame(post_hoc_fold_change_dict)

In [ ]:
post_hoc_fold_change_df

In [ ]:
diffexp_fold_change_ph={}
  
for condition in post_hoc_diff_dict:

    diffexp_fold_change_ph[condition]={
        'up':[],
        'dn':[]
    }
    for transcript in post_hoc_diff_dict[condition]:
        
        #Note that this fold change is for condition 2 relative to condition 1 in 1_2
        fold_change=post_hoc_fold_change_df.loc[transcript, condition]
        if fold_change > 2:
            diffexp_fold_change_ph[condition]['up'].append(transcript)

        elif fold_change < 0.5:
            diffexp_fold_change_ph[condition]['dn'].append(transcript)


In [ ]:
#Change probe names to gene names
diffexp_genes_fold_change_ph={}

for i in diffexp_fold_change_ph:
    
    diffexp_genes_fold_change_ph[i]={}
    
    for j in diffexp_fold_change_ph[i]:
        
        diffexp_genes_fold_change_ph[i][j]=set()
        
        for probe in diffexp_fold_change_ph[i][j]:
            
            if type(probe_key[probe])==list:
                
                for gene in probe_key[probe]:
                    diffexp_genes_fold_change_ph[i][j].add(gene)

In [ ]:
for condition in diffexp_genes_fold_change_ph:
    for up_or_dn in diffexp_genes_fold_change_ph[condition]:
        print('{}_{}: {} genes'.format(condition, up_or_dn, len(diffexp_genes_fold_change_ph[condition][up_or_dn])))

## Compare with gene sets

In [ ]:
gene_sets

In [ ]:
for gene_set in gene_sets:
    condition=gene_sets[gene_set].split('_')
    up_or_dn=condition[2]
    
    #Only comparing fold change from control
    if condition[1] == 'Control':
        gene_set_genes=set(gene_sets_dict[gene_set])
        diff_exp_genes=set(diffexp_genes_fold_change_ph['{}_{}'.format(condition[1], condition[0])][up_or_dn])
        print(gene_sets[gene_set])
        fig=plt.figure()
        ax=plt.gca()
        venn2([gene_set_genes, diff_exp_genes], set_labels=[gene_set, 'our genes'], ax=ax)

# Compare ANOVA with no post-hoc analysis to ANVOA with post-hoc analysis (Remove ?)

In [ ]:
for comparison in diffexp_fold_change_ph:
    condition=comparison.split('_')
    if condition[0]=='Control':
        for up_or_dn in ['up', 'dn']:
        
            deg_a=set(diffexp_fold_change[condition[1]][up_or_dn])
            deg_b=set(diffexp_fold_change_ph[comparison][up_or_dn])
            
            print('{}_{}'.format(comparison, up_or_dn))
            
            fig=plt.figure()
            ax=plt.gca()
            venn2([deg_a, deg_b], set_labels=['No post-hoc', 'With post-hoc'], ax=ax)            

# Create Volcano Plots (Using post-hoc p-values)

In [ ]:
#These volcano plots look unusual because we have only plotted genes we have already determined to be significant 
#through Tukey.

for condition in post_hoc_fold_change_df:
    
    volcano_plot_df=pd.DataFrame([pd.Series(-np.log10(post_hoc_fold_change_df[condition]), name='-log(Fold Change)'), 
                                  pd.Series(-np.log10(post_hoc_diff_df[condition]), name='-log(p)')]
                                 )
    
    volcano_plot_df.columns=[probe_key[i] for i in volcano_plot_df.columns]
    
    volcano_plot_df=volcano_plot_df.T
    
    volcano_plot_df['hover_text_labels']=volcano_plot_df.index
    
    fig=px.scatter(volcano_plot_df, x='-log(Fold Change)', y='-log(p)', hover_data=['hover_text_labels'], title=condition)
    fig.show()

# Probe IL2/15/21 Essentiality in DepMap/CCLE

In [ ]:
dm_sample_info=pd.read_csv('../data/dm_sample_info.csv', index_col=0)
dm_gene_expression=pd.read_csv('../data/CCLE_expression.csv', index_col=0)

In [ ]:
cks=['IL2', 'IL15', 'IL21']

In [ ]:
lineage=dm_sample_info['lineage']

In [ ]:
dm_gene_expression.columns=[i.split(' ')[0] for i in dm_gene_expression.columns]

In [ ]:
dm_gene_expression_oi=dm_gene_expression.loc[:,cks]

In [ ]:
dependency=pd.read_csv('../data/CRISPR_gene_dependency.csv', index_col=0)

In [ ]:
dependency

### Make one histogram with all cytokines

In [ ]:
#append values into a format plotly express will take

values=[]
labels=[]
index=[]

for ck in cks:
    values+=[*dm_gene_expression[ck].values]
    labels+=[ck]*dm_gene_expression[ck].shape[0]

In [ ]:
big_px_df=pd.DataFrame([values, labels], index=['gene_expression', 'gene'], dtype=float)

In [ ]:
px.histogram(data_frame=big_px_df.T, x='gene_expression', color='gene', barmode='overlay', title="Average Cytokine Gene Expression")

In [ ]:
dm_gene_expression

In [ ]:
dm_ge_x_lineage=dm_gene_expression.groupby(lineage).mean()

In [ ]:
values=[]
labels=[]
index=[]

for ck in cks:
    values+=[*dm_ge_x_lineage[ck].values]
    labels+=[ck]*dm_ge_x_lineage[ck].shape[0]
    index+=[*dm_ge_x_lineage.index]

In [ ]:
big_px_df=pd.DataFrame([values, labels, index], index=['average_gene_expression', 'gene', 'tissue_lineage'], dtype=float)

In [ ]:
px.bar(data_frame=big_px_df.T, x='tissue_lineage', y='average_gene_expression', color='gene', barmode='overlay', title='Average Cytokine Gene Expression by Tissue Lineage', height=1000)

In [ ]:
#append values into a format plotly express will take

values=[]
labels=[]
index=[]

for ck in cks:
    values+=[*dependency[ck].values]
    labels+=[ck]*dependency[ck].shape[0]

In [ ]:
big_px_df=pd.DataFrame([values, labels], index=['gene_dependency_score', 'gene'], dtype=float)

In [ ]:
px.histogram(data_frame=big_px_df.T, x='gene_dependency_score', color='gene', barmode='overlay', title='Cytokine Gene Dependency Score')

In [ ]:
dm_dep_x_lineage=dependency.groupby(lineage).mean()

In [ ]:
values=[]
labels=[]
index=[]

for ck in cks:
    values+=[*dm_dep_x_lineage[ck].values]
    labels+=[ck]*dm_dep_x_lineage[ck].shape[0]
    index+=[*dm_dep_x_lineage.index]

In [ ]:
big_px_df=pd.DataFrame([values, labels, index], index=['average_gene_dependency', 'gene', 'tissue_lineage'], dtype=float)

In [ ]:
px.bar(data_frame=big_px_df.T, x='tissue_lineage', y='average_gene_dependency', color='gene', barmode='overlay', title='Average Cytokine Gene  Dependency Score by Tissue Lineage', height=1000)

### or make 3 different histograms - to see different things

In [ ]:
dm_gene_expression=dm_gene_expression.T

In [ ]:
for ck in cks:

    fig=px.histogram(data_frame=dm_gene_expression.loc[ck].to_frame(), x=ck, range_x=[0,8], nbins=200, title='{}: {}'.format(ck, 'Gene Expression'), labels={ck: 'Gene Expression Value'})

    fig.show()
    
    fig=px.bar(dm_gene_expression.T.groupby(lineage).mean().loc[:, ck], title='Average {} Expression by Tissue Lineage'.format(ck))
    fig.show()

In [ ]:
dependency=dependency.T

In [ ]:
for ck in cks:

    fig=px.histogram(x=dependency.loc[ck].values, range_x=[0,1], nbins=200, title='{}: {}'.format(ck, 'Dependency'))

    fig.show()
    
    fig=px.bar(dependency.T.groupby(lineage).mean().loc[:, ck], title='Average {} Dependency by Tissue Lineage'.format(ck))
    fig.show()